## Dữ Liệu Chạy Chương Trình

In [1]:
import pandas as pd
import numpy as np
df_customer = pd.read_csv('customer_profile.csv')
df_consumption = pd.read_csv('consumption_log.csv')
df_tariff = pd.read_csv('price_tariff.csv')
print(f"Đã tải thành công 3 bảng dữ liệu: Khách hàng ({len(df_customer)} dòng), Tiêu thụ ({len(df_consumption)} dòng), Bảng giá ({len(df_tariff)} dòng).")
df_customer_clean = df_customer
print("\n[customer_profile - 5 dòng đã làm sạch]:")
print(df_customer_clean.head())
df_consumption['consumption_kwh'] = (df_consumption['consumption_kwh']
                                     .astype(str)
                                     .str.replace(r'[^\d\.-]', '', regex=True) # Chỉ giữ số, dấu chấm, dấu trừ
                                     .str.strip()
                                    )
df_consumption['consumption_kwh'] = pd.to_numeric(df_consumption['consumption_kwh'], errors='coerce')
df_consumption.loc[df_consumption['consumption_kwh'] < 0, 'consumption_kwh'] = np.nan
df_consumption_clean = df_consumption.dropna(subset=['reading_date', 'consumption_kwh']).reset_index(drop=True)
print("\n[consumption_log - 5 dòng đã làm sạch]:")
print(df_consumption_clean.head())
df_tariff.loc[df_tariff['tariff_level'] == 'B5', 'kwh_max'] = np.inf
df_tariff_clean = df_tariff.dropna(subset=['kwh_min', 'price_per_kwh']).reset_index(drop=True)
print("\n[price_tariff - Bảng giá đã làm sạch]:")
print(df_tariff_clean)
print("\n--- Xuất file sạch (Clean Files) ---")
df_customer_clean.to_csv('customer_profile_clean.csv', index=False)
print("-> Đã xuất file: customer_profile_clean.csv")
df_consumption_clean.to_csv('consumption_log_clean.csv', index=False)
print("-> Đã xuất file: consumption_log_clean.csv")
df_tariff_clean.to_csv('price_tariff_clean.csv', index=False)
print("-> Đã xuất file: price_tariff_clean.csv")
print("\n Hoàn thành việc làm sạch và xuất file.")

Đã tải thành công 3 bảng dữ liệu: Khách hàng (40 dòng), Tiêu thụ (150 dòng), Bảng giá (5 dòng).

[customer_profile - 5 dòng đã làm sạch]:
  customer_id house_type    region
0      KH100    Chung cu    ha noi
1      KH101   Biệt thự     ha noi
2       KH102  chung cư     Hà Nội
3      KH103     Tập thể    TP.HCM
4       KH104    Tập thể  Bac Ninh

[consumption_log - 5 dòng đã làm sạch]:
  customer_id meter_id reading_date  consumption_kwh
0       kh136   ct016    27/01/2024             50.0
1       kh119   ct010    2024-03-25             75.0
2       KH126    CT999   06/03/2024            150.0
3       kh135    CT999   2024-03-31             75.0
4       kh136    CT999   2024-04-15            300.0

[price_tariff - Bảng giá đã làm sạch]:
  tariff_level  kwh_min kwh_max price_per_kwh
0           B1        0      50          1800
1           B2       51     100         2000 
2           B3      101  200kwh          2500
3           B4      201     300         3000 
4           B5      301

#### Nv2

## Thống kê số hộ khách hàng theo house_type.

In [2]:
# Tái sử dụng các DataFrame đã làm sạch từ Nhiệm vụ 1
df_customer_clean = df_customer_clean
df_consumption_clean = df_consumption_clean

# Hợp nhất dữ liệu Tiêu thụ và Hồ sơ Khách hàng (Left Join)
df_merged = pd.merge(df_consumption_clean, df_customer_clean, on='customer_id', how='left')

print("Đã hoàn thành hợp nhất dữ liệu cho Nhiệm vụ 2.")
print(f"Số lượng bản ghi sau khi hợp nhất: {len(df_merged)}")
print("Các trường dữ liệu khả dụng: ['customer_id', 'reading_date', 'consumption_kwh', 'house_type', 'region', ...]")


#Thống kê Số hộ Khách hàng theo Loại nhà
print("\n--- 2.2. Số lượng Khách hàng theo Loại nhà ---")
house_type_count = df_customer_clean['house_type'].value_counts().reset_index()
house_type_count.columns = ['Loại Nhà', 'Số Lượng Khách Hàng']
total_customers = house_type_count['Số Lượng Khách Hàng'].sum()
house_type_count['Tỷ Lệ (%)'] = (house_type_count['Số Lượng Khách Hàng'] / total_customers * 100).round(1)
print(house_type_count)
#Tính Tổng và Trung bình

Đã hoàn thành hợp nhất dữ liệu cho Nhiệm vụ 2.
Số lượng bản ghi sau khi hợp nhất: 135
Các trường dữ liệu khả dụng: ['customer_id', 'reading_date', 'consumption_kwh', 'house_type', 'region', ...]

--- 2.2. Số lượng Khách hàng theo Loại nhà ---
    Loại Nhà  Số Lượng Khách Hàng  Tỷ Lệ (%)
0   tap the                     8       20.0
1    Tập thể                    7       17.5
2   biet thu                    7       17.5
3   Chung cu                    6       15.0
4  Biệt thự                     6       15.0
5  chung cư                     3        7.5
6   Nhà phố                     2        5.0
7    NHA pho                    1        2.5


##  Tính tổng và trung bình consumption_kwh theo từng khách hàng

In [3]:
print("\n--- 2.3. Tổng và Trung bình Điện năng tiêu thụ (kWh) theo Khách hàng (Top 5) ---")
customer_consumption_summary = df_consumption_clean.groupby('customer_id')['consumption_kwh'].agg(
    Tổng_Tiêu_Thụ=('sum'),
    Trung_Bình_Tiêu_Thụ=('mean')
).reset_index()
customer_consumption_summary['Trung_Bình_Tiêu_Thụ'] = customer_consumption_summary['Trung_Bình_Tiêu_Thụ'].round(2)
print(customer_consumption_summary.sort_values(by='Tổng_Tiêu_Thụ', ascending=False).head(5))


--- 2.3. Tổng và Trung bình Điện năng tiêu thụ (kWh) theo Khách hàng (Top 5) ---
   customer_id  Tổng_Tiêu_Thụ  Trung_Bình_Tiêu_Thụ
38       kh138         1340.0               167.50
17       KH107         1210.0               151.25
10       KH130         1050.0               210.00
33       kh115          890.0               178.00
37       kh136          890.0               178.00


## Tìm các bản ghi consumption_kwh bất thường (âm, bằng 0 hoặc quá lớn)

In [4]:
#Phân tích Bản ghi Tiêu thụ Bất thường
# Giá trị bằng 0
print("\n--- 2.4. Phân tích Bản ghi Tiêu thụ Bất thường ---")
zero_consumption = df_consumption_clean[df_consumption_clean['consumption_kwh'] == 0]
if not zero_consumption.empty:
    print(f"Số bản ghi có tiêu thụ bằng 0 kWh: {len(zero_consumption)}")
    print("Các bản ghi này cần được kiểm tra để xác định liệu đây có phải là lỗi ghi số hay do khách hàng ngưng sử dụng trong kỳ.")
else:
    print("Không tìm thấy bản ghi có consumption_kwh = 0 (sau khi làm sạch).")
#Giá trị Quá lớn
Q1 = df_consumption_clean['consumption_kwh'].quantile(0.25)
Q3 = df_consumption_clean['consumption_kwh'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
outlier_consumption = df_consumption_clean[df_consumption_clean['consumption_kwh'] > upper_bound]
print(f"\nNgưỡng xác định Outlier (Q3 + 1.5*IQR): {upper_bound:.2f} kWh")
print(f"Số bản ghi được coi là Tiêu thụ BẤT THƯỜNG (quá lớn): {len(outlier_consumption)}")
print("[5 bản ghi tiêu thụ lớn nhất được xác định là Outlier]:")
print(outlier_consumption[['customer_id', 'reading_date', 'consumption_kwh']].sort_values(by='consumption_kwh', ascending=False).head())


--- 2.4. Phân tích Bản ghi Tiêu thụ Bất thường ---
Không tìm thấy bản ghi có consumption_kwh = 0 (sau khi làm sạch).

Ngưỡng xác định Outlier (Q3 + 1.5*IQR): 262.50 kWh
Số bản ghi được coi là Tiêu thụ BẤT THƯỜNG (quá lớn): 26
[5 bản ghi tiêu thụ lớn nhất được xác định là Outlier]:
   customer_id reading_date  consumption_kwh
4        kh136   2024-04-15            300.0
10       kh115   2024-03-12            300.0
14       KH104   2024-03-25            300.0
17       kh138   2024-04-16            300.0
24       KH107   19/02/2024            300.0
